<a href="https://colab.research.google.com/github/ChabiMax/author_network/blob/main/Scraping%20propre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center><strong><font color="chillipepper">Réseau Wikipédia des auteurs francophones</font></strong></center></h1>
<h5><center><font color="green">Science des réseaux sociaux et économiques <h3><center>


<center>
Maxime Chabriel, Yasmine Houri, Mathis Sansu  

ENSAE 2022
</center>

<h2><center><strong><font color="chillipepper">Scraping des données</font></strong></center></h2>

# Packages

In [1]:
from tqdm import *
import time
import requests
import bs4
from collections import Counter
import json
import os
from google.colab import files
import pickle as pickle
import pandas as pd

# Scraping

In [2]:
raw_data = {} # Dictionnaire pour stocker les données

In [3]:
# URL de la page initiale de notre liste d'auteurs
URL = 'https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_chronologique'
response = requests.get(URL) # Go to the page
soup = bs4.BeautifulSoup(response.text, 'html.parser') # Récupération du code html

In [4]:
dictionary = {} # Déclaration d'un dictionnaire vide
# On initialise des attributs
date_naissance = None
date_mort = None
name = 'init'
author_link = 'init'
id = 0
serie = 0
links = soup.find(id="bodyContent").select('a') # Récupération des hyperliens

In [5]:
# Extraction de la liste d'auteurs

for link in links: # Boucle sur les hyperliens récupérés
  string = link.get_text()
  if link.get('href') != None:
    if not any(char.isdigit() for char in string) : 
      if serie == 2 :
        serie = 0
        date_mort = None
        dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}
      serie = serie+1
      id = id+1
      name = string
      date_naissance = None
      date_mort = None
      if 'https://' in link.get('href'):
        author_link = link.get('href')
      else:
        author_link = 'https://fr.wikipedia.org' + link.get('href')
    else :
      if serie == 1 :
        serie = serie+1
        date_naissance = string
      else :
        if serie == 2 :
          serie = 0
          date_mort = string
          dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}

In [6]:
# Nettoyage manuel : certains lien récupérés ne correpondent pas à des auteurs
unwanted_list = [key for key in dictionary.keys() if 'modifier' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'siècle' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'ms.' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'Portail' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Liste d'écrivains de langue française" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "↑" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "" == dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Langue française et francophonie" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Chronologie de la littérature" in dictionary[key]['name']]
# Les commandes au-dessus permettent de récupérer les clés problématiques dans le dictionnaire
# On enlève donc les éléments associés à ces clés dans le dictionnaire
dictionary = dict((key, dictionary[key]) for key in dictionary.keys() if key not in unwanted_list)

In [7]:
 # Extraction de la liste des genres littéraires (mêmes procédures)

URL = 'https://fr.wikipedia.org/wiki/Genre_litt%C3%A9raire'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary_genres = {}
name = 'init'
genre_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    id = id+1
    name = string
    if 'https://' in link.get('href'):
      genre_link = link.get('href')
    else:
      genre_link = 'https://fr.wikipedia.org' + link.get('href')
    dictionary_genres[id] = {'name' : name, 'link' : genre_link}


In [8]:
# Nettoyage manuel
unwanted_list = list(range(0,49))
unwanted_list += list(range(278,344))
unwanted_list += [key for key in dictionary_genres.keys() if 'modifier' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if dictionary_genres[key]['link'] in [dictionary[key_bis]['link'] for key_bis in dictionary.keys()]]
unwanted_list += [key for key in dictionary_genres.keys() if any(char.isdigit() for char in dictionary_genres[key]['name'])]
unwanted_list += list(range(285,len(dictionary_genres.keys())))
dictionary_genres = dict((key, dictionary_genres[key]) for key in dictionary_genres.keys() if key not in unwanted_list)

In [9]:
# Reset des keys 
old_dictionary = dictionary.copy()
dictionary = {}
id = 0
for key in old_dictionary :
  dictionary[id] = old_dictionary[key]
  id += 1

old_dictionary_genres = dictionary_genres.copy()
dictionary_genres = {}
for key in old_dictionary_genres :
  dictionary_genres[id] = old_dictionary_genres[key]
  id += 1

In [10]:
# Fonction réduisant une balise html de lien sous forme d'un lien wikipédia classique
def html_to_link(html_link) :
  if html_link.get('href') != None :
    if 'https://' in html_link.get('href'):
      html_link = html_link.get('href')
    else:
      html_link = 'https://fr.wikipedia.org' + html_link.get('href')
  else :
    html_link = ''
  return(html_link)

In [11]:
# Création des listes des liens associés à nos pages Wikipédia (auteurs et genres)
author_links = [dictionary[key]['link'] for key in dictionary.keys()]
genre_links = [dictionary_genres[key]['link'] for key in dictionary_genres.keys()]

In [12]:
# Parcours des pages et création du réseau
for id in tqdm_notebook(dictionary.keys()) :
  authtor_link = dictionary[id]['link']
  response = requests.get(authtor_link)
  soup = bs4.BeautifulSoup(response.text, 'html.parser')
  links = soup.find(id="bodyContent").select('a')
  network_aslink = []
  net_genre_aslink = []
  for link in links:
    link = html_to_link(link)
    if link in author_links :
      network_aslink.append(link)
    if link in genre_links :
      net_genre_aslink.append(link)
  network = []
  net_genre = []
  for link in network_aslink :
    for id_bis in dictionary.keys() :
      if link == dictionary[id_bis]['link'] :
        network.append(id_bis)
  for link in net_genre_aslink :
    for id_bis in dictionary_genres.keys() :
      if link == dictionary_genres[id_bis]['link'] :
        net_genre.append(id_bis)
  counted_network = Counter(network)
  counted_net_genre = Counter(net_genre)
  dictionary[id]['taille_html_article'] = len(str(soup))
  dictionary[id]['network_interauteur'] = list(counted_network.keys())
  dictionary[id]['poids_liens_interauteur'] = list(counted_network.values())
  dictionary[id]['genres_mentionnes'] = list(counted_net_genre.keys())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1584 [00:00<?, ?it/s]

# Formatage données

In [50]:
raw_data1 = dictionary.copy()
for key in raw_data1.keys() :
  raw_data1[key]['type'] = 'auteur'

for key in dictionary_genres.keys() :
  raw_data1[key] = dictionary_genres[key].copy()
  raw_data1[key]['type'] = 'genre'

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['auteurs_du_genre'] = []
for key_auteur in raw_data1.keys() :
  if raw_data1[key_auteur]['type'] == 'auteur' :
    for genre in raw_data1[key_auteur]['genres_mentionnes'] :
      raw_data1[genre]['auteurs_du_genre'] = raw_data1[genre]['auteurs_du_genre'] + [key_auteur]

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'auteur' :
    try :
      raw_data1[key_genre]['date_naissance'] = int(raw_data1[key_genre]['date_naissance'])
    except :
      raw_data1[key_genre]['date_naissance'] = 9999
    try :
      raw_data1[key_genre]['date_mort'] = int(raw_data1[key_genre]['date_mort'])
    except :
      raw_data1[key_genre]['date_mort'] = 9999

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['debut_genre'] = min([raw_data1[auteur]['date_naissance'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur' and key_genre in raw_data1[auteur]['genres_mentionnes']] + [9999])
    raw_data1[key_genre]['fin_genre'] = max([raw_data1[auteur]['date_mort'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur' and key_genre in raw_data1[auteur]['genres_mentionnes']] + [0])

for key_auteur1 in raw_data1.keys() :
  if raw_data1[key_auteur1]['type'] == 'auteur' :
    raw_data1[key_auteur1]['contemporain'] = []
    raw_data1[key_auteur1]['poids_contempo_diff_naissance'] = []
    raw_data1[key_auteur1]['poids_contempo_annees_communes'] = []
    for key_auteur2 in raw_data1.keys() :
      if (raw_data1[key_auteur2]['type'] == 'auteur') & (key_auteur2 != key_auteur1) :
        mort_1 = [raw_data1[key_auteur1]['date_mort'] if raw_data1[key_auteur1]['date_mort'] != None else 9999]
        mort_2 = [raw_data1[key_auteur2]['date_mort'] if raw_data1[key_auteur2]['date_mort'] != None else 9999]
        naissance_1 = [raw_data1[key_auteur1]['date_naissance'] if raw_data1[id]['date_mort'] != None else 0]
        naissance_2 = [raw_data1[key_auteur2]['date_naissance'] if raw_data1[key_auteur2]['date_mort'] != None else 0]
        try:
          annee_communes = [annee for annee in list(range(naissance_1, mort_1)) if annee in list(range(naissance_2, mort_2))]
          if (mort_1 > naissance_2 and naissance_1 < mort_2) or (mort_2 > naissance_1 and naissance_2 < mort_1):
            raw_data1[key_auteur1]['contemporain'].append(key_auteur2)
            raw_data1[key_auteur1]['poids_contempo_diff_naissance'].append(abs(naissance_1 - naissance_2))
            raw_data1[key_auteur1]['poids_contempo_annees_communes'].append(len(annee_communes))

        except :
          pass


In [51]:
df = pd.DataFrame(raw_data1).T
df.tail(50)

,name,link,date_mort,date_naissance,taille_html_article,network_interauteur,poids_liens_interauteur,genres_mentionnes,type,contemporain,poids_contempo_diff_naissance,poids_contempo_annees_communes,auteurs_du_genre,debut_genre,fin_genre
1743,maxime,https://fr.wikipedia.org/wiki/Maxime_(langue),NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,"[393, 767, 792, 1072, 1253]",1809,2025
1744,paradoxe,https://fr.wikipedia.org/wiki/Paradoxe,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,"[172, 242, 295, 383, 393, 1018]",1601,2025
1745,greguería,https://fr.wikipedia.org/wiki/Greguer%C3%ADa,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1746,aphorisme,https://fr.wikipedia.org/wiki/Aphorisme,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,"[269, 313, 314, 379, 390, 393, 615, 754, 767, ...",1715,2025
1747,pensée(s),https://fr.wikipedia.org/wiki/Pens%C3%A9es_(ge...,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1748,adab,https://fr.wikipedia.org/wiki/Adab_(litt%C3%A9...,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1749,haïbun,https://fr.wikipedia.org/wiki/Ha%C3%AFbun,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1750,fragment,https://fr.wikipedia.org/wiki/Fragment,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,"[825, 1182, 1216, 1295, 1326, 1445]",1907,2025
1751,caractère,https://fr.wikipedia.org/wiki/Caract%C3%A8re_(...,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1752,portrait,https://fr.wikipedia.org/wiki/Portrait,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,"[172, 272, 714, 758, 1326]",1601,2025


In [32]:
df.date_naissance.isna()

0       False
1       False
2       False
3       False
4       False
        ...  
1788     True
1789     True
1790     True
1791     True
1792     True
Name: date_naissance, Length: 1793, dtype: bool

# Import des données

In [14]:
# # Import sous forme pickle (objet python assez léger)
# import pickle
# with open('raw_data.pkl', 'wb') as f:
#     pickle.dump(raw_data1, f)
# files.download('raw_data.pkl')